In [35]:
import mne
import numpy as np

import os

import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.utils import resample

In [36]:
directory = './eeg-during-mental-arithmetic-tasks-1.0.0/'

rest_filepaths = []
task_filepaths = []

for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    if filename.endswith('.edf'):
        label = filename.split('_')[-1].split('.')[0]

        if label == '1':
            rest_filepaths.append(filepath)
        else:
            task_filepaths.append(filepath)

In [37]:
def read_data(filepath):
    data = mne.io.read_raw_edf(filepath, preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=0.5, h_freq=45)
    tmin, tmax = 0, 61.99
    data.crop(tmin=tmin, tmax=tmax)
    

    # if array.shape[0]>121:
    #     array = resample(array, replace=False, n_samples=average_epochs, random_state=42)
    # else:
    #     # Oversample 'task' epochs to the average value
    #     array = resample(array, replace=True, n_samples=average_epochs, random_state=42)
    return data.get_data()

In [38]:
%%capture
rest_epochs_array = [read_data(filepath) for filepath in rest_filepaths]
task_epochs_array = [read_data(filepath) for filepath in task_filepaths]

In [39]:
rest_epochs_array = np.array(rest_epochs_array)
task_epochs_array = np.array(task_epochs_array)
rest_epochs_array.shape

(36, 21, 30996)

In [40]:
# Labels:
rest_label = [0 for _ in rest_epochs_array]
task_label = [1 for _ in task_epochs_array]

In [41]:
all_epochs = np.concatenate((rest_epochs_array, task_epochs_array))
all_labels = rest_label + task_label

In [42]:
perm = np.random.permutation(72)

# Shuffle both arrays using the same permutation along the first axis
shuffled_epochs = []
shuffled_labels = []

for index in perm:
    shuffled_epochs.append(all_epochs[index])
    shuffled_labels.append(all_labels[index])

all_epochs = shuffled_epochs
all_labels = shuffled_labels

In [43]:
len(all_epochs), len(all_epochs[0]), len(all_epochs[0][0])

(72, 21, 30996)

In [44]:
data_array = np.array(all_epochs)
label_array = np.array(all_labels)

In [45]:
print(data_array.shape, label_array.shape)

(72, 21, 30996) (72,)


In [46]:
# Normalize
import numpy as np
eeg_data= data_array


# Assuming your EEG data is stored in a variable called `eeg_data`
# Shape of eeg_data: (72, 21, 30951)

# Initialize an array to store the normalized data
normalized_data = np.zeros_like(eeg_data)

means = np.mean(eeg_data, axis=(0, 2))
stds = np.std(eeg_data, axis=(0, 2))

# Normalize each channel for all candidates
for candidate in range(eeg_data.shape[0]):
    for channel in range(eeg_data.shape[1]):
        normalized_data[candidate, channel, :] = (eeg_data[candidate, channel, :] - means[channel]) / stds[channel]

# normalized_eeg_data now contains the z-score normalized EEG data


In [47]:
from tensorflow.keras.utils import to_categorical

label_array_one_hot = to_categorical(label_array)
label_array_one_hot.shape

(72, 2)

In [48]:
input_data_reshaped = tf.reshape(normalized_data, (-1, 21, 30996, 1))
input_data_reshaped.shape

TensorShape([72, 21, 30996, 1])

In [49]:
# Define Conv2D layer
input_shape = input_data_reshaped.shape
print("input shape: ", input_shape)

conv2d_layer = tf.keras.layers.Conv2D(filters=256, kernel_size=(1, 36), strides=(1, 36), padding='valid', input_shape=input_shape)

# Define BatchNormalization layer
batch_norm_layer = tf.keras.layers.BatchNormalization()

# Define DepthwiseConv2D layer
depthwise_conv2d_layer = tf.keras.layers.DepthwiseConv2D(kernel_size=(8, 1), strides=(8,1),padding='valid')

# Pass input_data through Conv2D layer
x = conv2d_layer(input_data_reshaped)
print("after conv2d  ", x.shape)

# Pass output through BatchNormalization layer
x = batch_norm_layer(x)
print("after batch  ", x.shape)

# Pass output through DepthwiseConv2D layer
output = depthwise_conv2d_layer(x)
print("after depthwise conv2d  ", output.shape)

np.savez('image_patches.npz', data=output)


input shape:  (72, 21, 30996, 1)
after conv2d   (72, 21, 861, 256)
after batch   (72, 21, 861, 256)
after depthwise conv2d   (72, 2, 861, 256)


# Run from here:


In [50]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Add, Dense, Dropout, Embedding, GlobalAveragePooling1D, Input, Layer, LayerNormalization, MultiHeadAttention

# ViT from scratch:


### All Functions:


In [51]:

class PatchEncoder(Layer):
    def __init__(self, num_patches=256, projection_dim=1722):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection_dim = projection_dim
        w_init = tf.random_normal_initializer()
        class_token = w_init(shape=(1, projection_dim), dtype="float32")
        self.class_token = tf.Variable(initial_value=class_token, trainable=True)
        self.projection = Dense(units=projection_dim)
        self.position_embedding = Embedding(input_dim=num_patches+1, output_dim=projection_dim)
        
    def call(self, patch):
        batch = tf.shape(patch)[0]
        # reshape the class token embedins
        class_token = tf.tile(self.class_token, multiples = [batch, 1])
        class_token = tf.reshape(class_token, (batch, 1, self.projection_dim))
        # calculate patches embeddings
        patches_embed = self.projection(patch)
        patches_embed = tf.concat([patches_embed, class_token], 1)
        # calcualte positional embeddings
        positions = tf.range(start=0, limit=self.num_patches+1, delta=1)
        positions_embed = self.position_embedding(positions)
        # add both embeddings
        encoded = patches_embed + positions_embed
        return encoded
        
        
class MLP(Layer):
    def __init__(self, hidden_features, out_features, dropout_rate=0.1, activation_func = None):
        super(MLP, self).__init__()
        self.dense1 = Dense(hidden_features, activation=tf.nn.gelu)
        self.dense2 = Dense(out_features, activation=activation_func)
        self.dropout = Dropout(dropout_rate)

    def call(self, x):
        x = self.dense1(x)
        x = self.dropout(x)
        x = self.dense2(x)
        y = self.dropout(x)
        return y
    
    
class Block(Layer):
    def __init__(self, projection_dim, num_heads=4, dropout_rate=0.1):
        super(Block, self).__init__()
        self.norm1 = LayerNormalization(epsilon=1e-6)
        self.attn = MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim, dropout=dropout_rate)
        self.norm2 = LayerNormalization(epsilon=1e-6)
        self.mlp = MLP(projection_dim * 2, projection_dim, dropout_rate)

    def call(self, x):
        # Layer normalization 1.
        x1 = self.norm1(x) # encoded_patches
        # Create a multi-head attention layer.
        attention_output = self.attn(x1, x1)
        # Skip connection 1.
        x2 = Add()([attention_output, x]) #encoded_patches
        # Layer normalization 2.
        x3 = self.norm2(x2)
        # MLP.
        x3 = self.mlp(x3)
        # Skip connection 2.
        y = Add()([x3, x2])
        return y
    
class TransformerEncoder(Layer):
    def __init__(self, projection_dim, num_heads=4, num_blocks=12, dropout_rate=0.1):
        super(TransformerEncoder, self).__init__()
        self.blocks = [Block(projection_dim, num_heads, dropout_rate) for _ in range(num_blocks)]
        self.norm = LayerNormalization(epsilon=1e-6)
        self.dropout = Dropout(dropout_rate)

    def call(self, x):
        # Create a [batch_size, projection_dim] tensor.
        for block in self.blocks:
            x = block(x)
        x = self.norm(x)
        y = self.dropout(x)
        return y

In [58]:
# Example array with shape (72, 2, 861, 256)
original_array = np.zeros((72, 2, 861, 256))

# Calculate total number of elements
total_elements = np.prod(original_array.shape)

# Calculate a such that 72 * a * a = total_elements / 2
a = int(np.sqrt(total_elements / (72 * 2)))

# Adjust a to be a little smaller to avoid size mismatches
a = min(a, 861)  # Choose a maximum value that ensures reshaping works

# Reshape the array to (72, a, a)
reshaped_array = np.reshape(original_array, (72, a, a))

# Print shapes for verification
print("Original Shape:", original_array.shape)
print("Reshaped Shape:", reshaped_array.shape)

ValueError: cannot reshape array of size 31739904 into shape (72,469,469)

In [54]:
def create_VisionTransformer(num_classes, num_patches=256, projection_dim=182, input_shape=(256, 182)):
    patches = Input(shape=input_shape)
    # Patch encoder
    patches_embed = PatchEncoder(num_patches, projection_dim)(patches)
    # Transformer encoder
    representation = TransformerEncoder(projection_dim)(patches_embed)
    representation = GlobalAveragePooling1D()(representation)
    # MLP to classify outputs
    logits = MLP(projection_dim, num_classes, 0.5, 'sigmoid')(representation)
    # Create model
    model = Model(inputs=patches, outputs=logits)
    return model

In [ ]:
model = create_VisionTransformer(2, input_shape=(256, 182))

In [ ]:
from sklearn.model_selection import GroupKFold

num_classes = 1  # For binary classification with sigmoid, num_classes should be 1



gkf = GroupKFold(n_splits=4)


In [ ]:
patches = np.moveaxis(patches, 1, 2)
patches.shape

(8784, 182, 256)

In [ ]:

from tensorflow.keras.losses import BinaryCrossentropy
test_accuracy = []

for train_index, test_index in gkf.split(patches, label_array, groups=group_array):
    # Split data into train and test for this fold
    train_features, test_features = patches[train_index], patches[test_index]
    train_labels, test_labels = label_array[train_index], label_array[test_index]

    train_features = np.moveaxis(train_features, 1, 2)
    test_features = np.moveaxis(test_features, 1, 2)

    train_features = np.reshape(train_features, (*train_features.shape, 1))
    test_features = np.reshape(test_features, (*test_features.shape, 1))

    # Initialize model

    model = create_VisionTransformer(num_classes, input_shape=(256, 182))

    # Compile model
    model.compile(optimizer='adam', loss=BinaryCrossentropy(),
                  metrics=['accuracy'])

    # Train model on current fold's train and validation data

    model.fit(train_features, train_labels, epochs=10,
              batch_size=128, validation_split=0.2)

    # Evaluate model on test data for this fold
    test_loss, test_acc = model.evaluate(test_features, test_labels)
    test_accuracy.append(test_acc)


# After all folds, print average test accuracy


print("Average Test Accuracy:", np.mean(test_accuracy))

Epoch 1/10
11/42 ━━━━━━━━━━━━━━━━━━━━ 8:50 17s/step - accuracy: 0.4920 - loss: 7.3537

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(patches, label_array, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 616s 5s/step - accuracy: 0.4924 - loss: 8.0326 - val_accuracy: 0.7223 - val_loss: 1.5540
Epoch 2/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 575s 5s/step - accuracy: 0.5167 - loss: 7.7444 - val_accuracy: 0.7223 - val_loss: 1.5540
Epoch 3/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 566s 5s/step - accuracy: 0.5047 - loss: 7.9345 - val_accuracy: 0.7223 - val_loss: 1.5540
Epoch 4/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 561s 5s/step - accuracy: 0.5035 - loss: 7.9514 - val_accuracy: 0.7223 - val_loss: 1.5540
Epoch 5/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 561s 5s/step - accuracy: 0.5062 - loss: 7.9095 - val_accuracy: 0.7223 - val_loss: 1.5540
Epoch 6/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 563s 5s/step - accuracy: 0.5065 - loss: 7.9069 - val_accuracy: 0.7223 - val_loss: 1.5540
Epoch 7/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 560s 5s/step - accuracy: 0.5054 - loss: 7.9244 - val_accuracy: 0.7223 - val_loss: 1.5540
Epoch 8/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 562s 5s/step - accuracy: 0.4966 - loss: 8.0638 - val_accu

In [ ]:
model.evaluate(x_test,y_test)

58/58 ━━━━━━━━━━━━━━━━━━━━ 57s 973ms/step - accuracy: 0.4866 - loss: 4.6618


[4.843190670013428, 0.46666666865348816]

In [ ]:
weights_path = 'model_weights.h5'

# Save the weights
model.save_weights(weights_path)

ValueError: The filename must end in `.weights.h5`. Received: filepath=model_weights.h5